In [ ]:
# This code is meant to analyze the SWR passenger loadings data

In [2]:
import os
import sys
import pandas as pd
import numpy as np 

In [4]:
project_root = os.path.dirname(os.getcwd())
sys.path.insert(0, project_root)

from data.passenger_loadings import SWR_passenger_loadings_files

In [17]:
# Read the SWR passenger loadings data for P10 (the period used in the analysis)
# The actual data is in the 'Loadings Data' sheet
# The header has two rows: some columns span both rows, others have parent-child structure
SWR_data = pd.read_excel(SWR_passenger_loadings_files["24-25 P10"], sheet_name='Loadings Data', header=[0, 1])

# Flatten the multi-level column names properly
new_columns = []
for col in SWR_data.columns:
    # col is a tuple like ('Public Times', 'Arr') or ('Rail Period', nan)
    if pd.notna(col[1]) and 'Unnamed' not in str(col[1]):
        # Second row has a meaningful name (like Arr, Dep, or the last 4 columns)
        new_columns.append(col[1])
    elif pd.notna(col[0]) and 'Unnamed' not in str(col[0]):
        # First row has the name
        new_columns.append(col[0])
    else:
        # Both are unnamed, keep as is for now
        new_columns.append(f"{col[0]}_{col[1]}")

SWR_data.columns = new_columns

SWR_data.columns.tolist()

['Rail Period',
 'TrainID',
 'Headcode',
 'Train UID',
 'Origin',
 'Destination',
 'Origin Time',
 'Destination Time',
 'Calendar Day',
 'Location',
 'Location Name',
 'Arr',
 'Dep',
 'Formation',
 'Car Count',
 'Standard Seated',
 'Standing',
 'Capacity',
 'First Class Seats',
 'Average Train Load on Departure',
 '% of Seated Capacity',
 '% of Total Capacity',
 'Number of Readings']

In [21]:
# Get unique locations from the data
SWR_locations = sorted(SWR_data['Location'].unique())

print(f"Number of unique locations: {len(SWR_locations)}")
print("\nUnique locations:", SWR_locations)

Number of unique locations: 199

Unique locations: ['ADLESTN', 'ALDRSHT', 'ALTON', 'ANDOVER', 'ASCOT', 'ASFDMSX', 'ASHD', 'ASHH', 'ASHVALE', 'AXMNSTR', 'BAGSHOT', 'BARNES', 'BDHMPTN', 'BITERNE', 'BKNHRST', 'BLIEURD', 'BNSBDGE', 'BNTEY', 'BNTFORD', 'BOMO', 'BOOKHAM', 'BOTLEY', 'BOXHAWH', 'BRACKNL', 'BRANKSM', 'BRKWOOD', 'BRLANDS', 'BRUTON', 'BSNGSTK', 'BURSLDN', 'BYFLANH', 'CBHMSDA', 'CBRK', 'CCARY', 'CHISWCK', 'CHRISTC', 'CHSSN', 'CHSSS', 'CHTSEY', 'CLANDON', 'CLPHMJM', 'CLPHMJW', 'CLYGATE', 'CMBLEY', 'COSHAM', 'CRKRN', 'CSFD', 'DATCHET', 'DEAN', 'DORKING', 'DRCHS', 'EFNGHMJ', 'EGHAM', 'ELGH', 'EPSM', 'ERLFLD', 'ERLY', 'ESHER', 'EWELW', 'EXETERC', 'EXETRSD', 'FAREHAM', 'FARNHAM', 'FELTHAM', 'FENITON', 'FLEET', 'FRATTON', 'FRBRMN', 'FRIMLEY', 'FRNCMB', 'FROME', 'FULWELL', 'GDLMING', 'GLHM', 'GRATELY', 'GUILDFD', 'HAMPTON', 'HAMWICK', 'HASLEMR', 'HAVANT', 'HCRT', 'HEDGEND', 'HERSHAM', 'HILSEA', 'HINTONA', 'HMBLE', 'HMWTHY', 'HNCHLYW', 'HOLTONH', 'HONITON', 'HOOK', 'HOUNSLW', 'HRSLEY', 'I

In [22]:
# Import reference files and load TIPLOC to STANOX mapping
import json
from data.reference import reference_files

# Load the reference data that contains TIPLOC to STANOX mapping
print("Loading reference data...")
with open(reference_files["all dft categories"], 'r') as f:
    reference_data = json.load(f)

# Convert to DataFrame
stanox_ref = pd.DataFrame(reference_data)

# Create TIPLOC to STANOX mapping dictionary
# Check which column names are used in the reference data
if 'tiploc' in stanox_ref.columns and 'stanox' in stanox_ref.columns:
    tiploc_to_stanox = dict(zip(stanox_ref['tiploc'], stanox_ref['stanox']))
elif 'tiploc_code' in stanox_ref.columns and 'stanox' in stanox_ref.columns:
    tiploc_to_stanox = dict(zip(stanox_ref['tiploc_code'], stanox_ref['stanox']))
else:
    print(f"Warning: Expected TIPLOC columns not found. Available columns: {list(stanox_ref.columns)}")
    tiploc_to_stanox = {}

print(f"Loaded {len(tiploc_to_stanox)} TIPLOC to STANOX mappings")

# Map STANOX codes to SWR locations
SWR_stanox = [tiploc_to_stanox.get(loc, None) for loc in SWR_locations]

# Create a DataFrame showing the mapping
location_mapping = pd.DataFrame({
    'TIPLOC': SWR_locations,
    'STANOX': SWR_stanox
})

print(f"\nLocations with STANOX codes:")
print(location_mapping)

Loading reference data...
Loaded 6104 TIPLOC to STANOX mappings

Locations with STANOX codes:
      TIPLOC STANOX
0    ADLESTN  86011
1    ALDRSHT  87031
2      ALTON  87021
3    ANDOVER  86074
4      ASCOT  87103
..       ...    ...
194  WRYSBRY  87108
195  WSORAER  87111
196  WSTBRYW  82101
197  YOVILJN  82341
198  YOVILPM  82311

[199 rows x 2 columns]


how many processed stanox do i have?

In [24]:
# Check which STANOX codes exist in the processed_data folder
processed_data_dir = os.path.join(project_root, 'processed_data')

# Get all folder names in processed_data (these should be STANOX codes)
if os.path.exists(processed_data_dir):
    existing_stanox_folders = [folder for folder in os.listdir(processed_data_dir) 
                               if os.path.isdir(os.path.join(processed_data_dir, folder))]
    
    # Convert to integers for comparison (STANOX codes are numeric)
    existing_stanox_set = set(existing_stanox_folders)
    
    # Check which SWR STANOX codes exist in processed_data
    # Remove None values from SWR_stanox
    swr_stanox_str = [str(stanox) for stanox in SWR_stanox if stanox is not None]
    
    # Find matches
    matching_stanox = [stanox for stanox in swr_stanox_str if stanox in existing_stanox_set]
    missing_stanox = [stanox for stanox in swr_stanox_str if stanox not in existing_stanox_set]
    
    print(f"Total SWR locations: {len(SWR_locations)}")
    print(f"Total folders in processed_data: {len(existing_stanox_folders)}")
    print(f"SWR locations with data in processed_data: {len(matching_stanox)}")
    print(f"SWR locations missing from processed_data: {len(missing_stanox)}")
    
    # Show which ones are available
    print(f"\n{len(matching_stanox)} SWR stations with processed data:")
    
    # Create a mapping to show TIPLOC names with their STANOX
    SWR_available_stations = location_mapping[location_mapping['STANOX'].astype(str).isin(matching_stanox)]
    print(SWR_available_stations.to_string())

else:
    print(f"processed_data directory not found at: {processed_data_dir}")

Total SWR locations: 199
Total folders in processed_data: 375
SWR locations with data in processed_data: 72
SWR locations missing from processed_data: 125

72 SWR stations with processed data:
      TIPLOC STANOX
1    ALDRSHT  87031
2      ALTON  87021
3    ANDOVER  86074
4      ASCOT  87103
5    ASFDMSX  87117
11    BARNES  87149
14   BKNHRST  86901
18   BNTFORD  87144
19      BOMO  86921
23   BRACKNL  87101
25   BRKWOOD  86041
28   BSNGSTK  86066
39   CLANDON  87069
49   DORKING  87687
52     EGHAM  87107
53      ELGH  86087
54      EPSM  87681
58     EWELW  87304
59   EXETERC  83431
60   EXETRSD  83421
61   FAREHAM  86241
62   FARNHAM  87026
63   FELTHAM  87121
65     FLEET  86045
67    FRBRMN  86042
72   GDLMING  87057
73      GLHM  82332
75   GUILDFD  87052
76   HAMPTON  87158
78   HASLEMR  87062
79    HAVANT  86341
80      HCRT  87181
82   HERSHAM  86003
97   LETHRHD  87685
109   MLFORD  87059
112  MOTSPRP  87301
113   MRTLKE  87139
115  NEWMLDN  87169
116  NMILTON  86911
117  NR

"SWR_available_stations" stores the available stations pre-processed and in SWR routes

In [29]:
# Check O-D pairs present in both SWR data and preprocessed data
# First, convert SWR TIPLOC codes to STANOX codes for comparison

# Add STANOX codes to SWR data for Origin and Destination
SWR_data['Origin_STANOX'] = SWR_data['Origin'].map(tiploc_to_stanox)
SWR_data['Destination_STANOX'] = SWR_data['Destination'].map(tiploc_to_stanox)

# Get unique O-D pairs from SWR data (using STANOX codes)
swr_od_pairs_stanox = SWR_data[['Origin', 'Origin_STANOX', 'Destination', 'Destination_STANOX']].dropna().drop_duplicates()
print(f"Total unique O-D pairs in SWR data (with valid STANOX codes): {len(swr_od_pairs_stanox)}")

# Initialize a list to store matching O-D pairs
matching_od_pairs = []
total_processed_od_pairs = 0
corrupted_files = []
processed_files = 0

# For each available SWR station, check its preprocessed data
for idx, row in SWR_available_stations.iterrows():
    stanox = str(row['STANOX'])
    tiploc = row['TIPLOC']
    
    # Path to the preprocessed data for this station
    station_dir = os.path.join(processed_data_dir, stanox)
    
    # Look for parquet files in the station directory
    if os.path.exists(station_dir):
        parquet_files = [f for f in os.listdir(station_dir) if f.endswith('.parquet')]
        
        for pq_file in parquet_files:
            try:
                # Read the preprocessed data using fastparquet engine
                preprocessed_data = pd.read_parquet(os.path.join(station_dir, pq_file), engine='fastparquet')
                processed_files += 1
                
                # Check if the required columns exist
                if 'PLANNED_ORIGIN_LOCATION_CODE' in preprocessed_data.columns and \
                   'PLANNED_DEST_LOCATION_CODE' in preprocessed_data.columns:
                    
                    # Get unique O-D pairs from preprocessed data (already in STANOX format)
                    processed_od = preprocessed_data[['PLANNED_ORIGIN_LOCATION_CODE', 
                                                       'PLANNED_DEST_LOCATION_CODE']].drop_duplicates()
                    total_processed_od_pairs += len(processed_od)
                    
                    # Check for matches with SWR O-D pairs (comparing STANOX to STANOX)
                    for _, swr_row in swr_od_pairs_stanox.iterrows():
                        swr_origin_tiploc = swr_row['Origin']
                        swr_origin_stanox = swr_row['Origin_STANOX']
                        swr_dest_tiploc = swr_row['Destination']
                        swr_dest_stanox = swr_row['Destination_STANOX']
                        
                        # Check if this O-D pair exists in the preprocessed data
                        match = processed_od[
                            (processed_od['PLANNED_ORIGIN_LOCATION_CODE'] == swr_origin_stanox) &
                            (processed_od['PLANNED_DEST_LOCATION_CODE'] == swr_dest_stanox)
                        ]
                        
                        if not match.empty:
                            matching_od_pairs.append({
                                'STANOX': stanox,
                                'TIPLOC': tiploc,
                                'Origin_TIPLOC': swr_origin_tiploc,
                                'Origin_STANOX': swr_origin_stanox,
                                'Destination_TIPLOC': swr_dest_tiploc,
                                'Destination_STANOX': swr_dest_stanox,
                                'File': pq_file
                            })
            except Exception as e:
                # Track corrupted files but continue processing
                corrupted_files.append({'STANOX': stanox, 'File': pq_file, 'Error': str(e)})

# Create DataFrame of matching O-D pairs
matching_od_df = pd.DataFrame(matching_od_pairs)

# Summary statistics
print(f"\nProcessing Summary:")
print(f"  Successfully processed files: {processed_files}")
print(f"  Corrupted/unreadable files: {len(corrupted_files)}")

# Get unique O-D pairs (might be same O-D pair in multiple stations)
if len(matching_od_df) > 0:
    unique_matching_od = matching_od_df[['Origin_STANOX', 'Destination_STANOX']].drop_duplicates()
    
    print(f"\nO-D Pair Analysis:")
    print(f"  Total unique O-D pairs in SWR data: {len(swr_od_pairs_stanox)}")
    print(f"  Total O-D pairs checked in preprocessed data: {total_processed_od_pairs}")
    print(f"  Unique matching O-D pairs found in both datasets: {len(unique_matching_od)}")
    print(f"  Total matches (including duplicates across stations): {len(matching_od_df)}")
    
    # Show summary by station
    station_summary = matching_od_df.groupby(['STANOX', 'TIPLOC']).size().reset_index(name='Num_Matching_OD_Pairs')
    print(f"\nMatching O-D pairs by station (top 20):")
    print(station_summary.nlargest(20, 'Num_Matching_OD_Pairs').to_string())
    
    print(f"\nSample of matching O-D pairs (first 20):")
    print(matching_od_df[['TIPLOC', 'Origin_TIPLOC', 'Origin_STANOX', 'Destination_TIPLOC', 'Destination_STANOX']].head(20).to_string())
else:
    print("\nNo matching O-D pairs found between SWR and preprocessed data")
    print("This might mean the O-D pairs in SWR data don't match any routes in the preprocessed data")

Total unique O-D pairs in SWR data (with valid STANOX codes): 183

Processing Summary:
  Successfully processed files: 504
  Corrupted/unreadable files: 0

O-D Pair Analysis:
  Total unique O-D pairs in SWR data: 183
  Total O-D pairs checked in preprocessed data: 22101
  Unique matching O-D pairs found in both datasets: 181
  Total matches (including duplicates across stations): 8596

Matching O-D pairs by station (top 20):
   STANOX   TIPLOC  Num_Matching_OD_Pairs
63  87261     WDON                    365
8   86031   WOKING                    332
60  87171  SURBITN                    323
59  87169  NEWMLDN                    310
62  87214  VAUXHLM                    269
12  86066  BSNGSTK                    249
53  87149   BARNES                    218
47  87131  TWCKNHM                    216
21  86520    SOTON                    214
15  86087     ELGH                    210
43  87114  STAINES                    205
10  86042   FRBRMN                    194
16  86122   SLSBRY       

In [41]:
# Check EVENT_DATETIME availability - properly adding STANOX from folder name
print("Checking EVENT_DATETIME availability (with STANOX from folder name):")
print("=" * 70)

# Get the matched O-D pairs from earlier analysis
matched_od_stanox_pairs = set(zip(matching_od_df['Origin_STANOX'], matching_od_df['Destination_STANOX']))

print(f"\nTotal unique matched O-D pairs: {len(matched_od_stanox_pairs)}")

# Statistics across all available stations
total_records = 0
records_with_datetime = 0
records_without_datetime = 0
stations_with_data = 0

# Check each available SWR station
for idx, row in SWR_available_stations.iterrows():
    stanox = str(row['STANOX'])
    tiploc = row['TIPLOC']
    
    station_dir = os.path.join(processed_data_dir, stanox)
    
    if os.path.exists(station_dir):
        parquet_files = [f for f in os.listdir(station_dir) if f.endswith('.parquet')]
        
        for pq_file in parquet_files:
            try:
                # Read the preprocessed data
                preprocessed_data = pd.read_parquet(os.path.join(station_dir, pq_file), engine='fastparquet')
                
                # ADD STANOX COLUMN FROM FOLDER NAME
                preprocessed_data['STANOX'] = int(stanox)
                
                # Check if required columns exist
                if all(col in preprocessed_data.columns for col in 
                       ['PLANNED_ORIGIN_LOCATION_CODE', 'PLANNED_DEST_LOCATION_CODE', 'EVENT_DATETIME']):
                    
                    # Filter to only matched O-D pairs
                    preprocessed_data['OD_pair'] = list(zip(
                        preprocessed_data['PLANNED_ORIGIN_LOCATION_CODE'], 
                        preprocessed_data['PLANNED_DEST_LOCATION_CODE']
                    ))
                    
                    matched_records = preprocessed_data[preprocessed_data['OD_pair'].isin(matched_od_stanox_pairs)]
                    
                    if len(matched_records) > 0:
                        stations_with_data += 1
                        total_records += len(matched_records)
                        
                        # Check which have EVENT_DATETIME
                        has_datetime = matched_records['EVENT_DATETIME'].notna()
                        records_with_datetime += has_datetime.sum()
                        records_without_datetime += (~has_datetime).sum()
                
            except Exception as e:
                pass

print(f"\nResults:")
print(f"  Stations with matched O-D data: {stations_with_data}")
print(f"  Total records with matched O-D pairs: {total_records}")

if total_records > 0:
    print(f"  Records WITH EVENT_DATETIME (had delays): {records_with_datetime} ({records_with_datetime/total_records*100:.1f}%)")
    print(f"  Records WITHOUT EVENT_DATETIME (no delays): {records_without_datetime} ({records_without_datetime/total_records*100:.1f}%)")
    
    if records_with_datetime > 0:
        print(f"\n✓ Good news! {records_with_datetime} train records have delay information (EVENT_DATETIME).")
        print(f"  We can use these for date-based matching with SWR data.")
        print(f"\n  The remaining {records_without_datetime} records without delays can still be matched")
        print(f"  using WEEKDAY for day-of-week patterns.")
    else:
        print(f"\n✗ All {records_without_datetime} matched train records have no delays (EVENT_DATETIME is None).")
        print(f"  We should use WEEKDAY column for day-of-week matching instead.")
else:
    print(f"\n✗ No records found with the matched O-D pairs.")

Checking EVENT_DATETIME availability (with STANOX from folder name):

Total unique matched O-D pairs: 181

Results:
  Stations with matched O-D data: 504
  Total records with matched O-D pairs: 1412645
  Records WITH EVENT_DATETIME (had delays): 284286 (20.1%)
  Records WITHOUT EVENT_DATETIME (no delays): 1128359 (79.9%)

✓ Good news! 284286 train records have delay information (EVENT_DATETIME).
  We can use these for date-based matching with SWR data.

  The remaining 1128359 records without delays can still be matched
  using WEEKDAY for day-of-week patterns.

Results:
  Stations with matched O-D data: 504
  Total records with matched O-D pairs: 1412645
  Records WITH EVENT_DATETIME (had delays): 284286 (20.1%)
  Records WITHOUT EVENT_DATETIME (no delays): 1128359 (79.9%)

✓ Good news! 284286 train records have delay information (EVENT_DATETIME).
  We can use these for date-based matching with SWR data.

  The remaining 1128359 records without delays can still be matched
  using WEEK

In [44]:
# Check the date range of preprocessed data (using correct format: DD-MMM-YYYY)
print("Checking date range of preprocessed data:")
print("=" * 70)

swr_start = pd.to_datetime('2024-12-08')
swr_end = pd.to_datetime('2025-01-04')

print(f"SWR data period: {swr_start.date()} to {swr_end.date()}")
print(f"Checking preprocessed data dates...\n")

all_dates = []
files_checked = 0

# Sample some files to check date range
for idx, row in SWR_available_stations.head(10).iterrows():
    stanox = str(row['STANOX'])
    tiploc = row['TIPLOC']
    
    station_dir = os.path.join(processed_data_dir, stanox)
    
    if os.path.exists(station_dir):
        parquet_files = [f for f in os.listdir(station_dir) if f.endswith('.parquet')]
        
        for pq_file in parquet_files[:2]:  # Check first 2 files per station
            try:
                preprocessed_data = pd.read_parquet(os.path.join(station_dir, pq_file), engine='fastparquet')
                
                if 'EVENT_DATETIME' in preprocessed_data.columns:
                    # Parse dates with correct format: DD-MMM-YYYY (e.g., "13-JUN-2024")
                    dates = pd.to_datetime(preprocessed_data['EVENT_DATETIME'], format='%d-%b-%Y', errors='coerce')
                    valid_dates = dates.dropna()
                    
                    if len(valid_dates) > 0:
                        all_dates.extend(valid_dates.tolist())
                        files_checked += 1
                        
                        if files_checked <= 3:  # Show details for first few files
                            print(f"{tiploc} ({pq_file}):")
                            print(f"  Date range: {valid_dates.min().date()} to {valid_dates.max().date()}")
                            print(f"  Sample dates: {[d.strftime('%d-%b-%Y') for d in valid_dates.head(3)]}")
                
            except Exception as e:
                pass

if all_dates:
    all_dates_series = pd.Series(all_dates)
    overall_min = all_dates_series.min()
    overall_max = all_dates_series.max()
    
    print(f"\n{'='*70}")
    print(f"Overall preprocessed data date range (from {files_checked} files):")
    print(f"  Earliest date: {overall_min.date()}")
    print(f"  Latest date: {overall_max.date()}")
    print(f"  Total span: {(overall_max - overall_min).days} days")
    
    # Check if SWR period is in the data
    dates_in_swr_period = all_dates_series[(all_dates_series >= swr_start) & (all_dates_series <= swr_end)]
    
    print(f"\nDates in SWR period (Dec 8, 2024 - Jan 4, 2025):")
    print(f"  Found: {len(dates_in_swr_period)} records")
    
    if len(dates_in_swr_period) > 0:
        print(f"  ✓ The SWR period IS covered in the preprocessed data!")
        print(f"  Date range in SWR period: {dates_in_swr_period.min().date()} to {dates_in_swr_period.max().date()}")
    else:
        print(f"  ✗ The SWR period is NOT in the preprocessed data.")
        print(f"  The data covers: {overall_min.date()} to {overall_max.date()}")
        print(f"\n  Since dates don't overlap, we'll use DAY-OF-WEEK matching instead.")
else:
    print("\nNo valid dates found in the sample. Most trains may not have had delays.")

Checking date range of preprocessed data:
SWR data period: 2024-12-08 to 2025-01-04
Checking preprocessed data dates...


No valid dates found in the sample. Most trains may not have had delays.


In [45]:
# More thorough check - look for files that have EVENT_DATETIME values
print("Finding date range from files with delay data:")
print("=" * 70)

all_dates_found = []
files_with_dates = 0
total_files_checked = 0

# Check all available SWR stations
for idx, row in SWR_available_stations.iterrows():
    stanox = str(row['STANOX'])
    tiploc = row['TIPLOC']
    
    station_dir = os.path.join(processed_data_dir, stanox)
    
    if os.path.exists(station_dir):
        parquet_files = [f for f in os.listdir(station_dir) if f.endswith('.parquet')]
        
        for pq_file in parquet_files:
            try:
                preprocessed_data = pd.read_parquet(os.path.join(station_dir, pq_file), engine='fastparquet')
                total_files_checked += 1
                
                if 'EVENT_DATETIME' in preprocessed_data.columns:
                    # Parse dates with format DD-MMM-YYYY
                    dates = pd.to_datetime(preprocessed_data['EVENT_DATETIME'], format='%d-%b-%Y', errors='coerce')
                    valid_dates = dates.dropna()
                    
                    if len(valid_dates) > 0:
                        all_dates_found.extend(valid_dates.tolist())
                        files_with_dates += 1
                        
                        # Stop after finding enough samples
                        if files_with_dates >= 20:
                            break
                            
            except Exception as e:
                pass
    
    if files_with_dates >= 20:
        break

print(f"Files checked: {total_files_checked}")
print(f"Files with valid dates: {files_with_dates}")
print(f"Total date records found: {len(all_dates_found)}")

if all_dates_found:
    dates_series = pd.Series(all_dates_found)
    
    print(f"\nDate range in preprocessed data:")
    print(f"  Earliest: {dates_series.min().date()} ({dates_series.min().strftime('%A')})")
    print(f"  Latest: {dates_series.max().date()} ({dates_series.max().strftime('%A')})")
    print(f"  Span: {(dates_series.max() - dates_series.min()).days} days")
    
    # Check overlap with SWR period
    swr_start = pd.to_datetime('2024-12-08')
    swr_end = pd.to_datetime('2025-01-04')
    
    in_swr_period = dates_series[(dates_series >= swr_start) & (dates_series <= swr_end)]
    
    print(f"\nSWR period: {swr_start.date()} to {swr_end.date()}")
    print(f"Records in SWR period: {len(in_swr_period)}")
    
    if len(in_swr_period) > 0:
        print(f"✓ SWR period IS covered! Can use exact date matching.")
        print(f"  Dates found: {in_swr_period.min().date()} to {in_swr_period.max().date()}")
    else:
        print(f"✗ SWR period NOT covered in preprocessed data.")
        print(f"  Will use WEEKDAY-based matching instead.")
        
    # Show date distribution by month
    print(f"\nDate distribution by month:")
    month_counts = dates_series.dt.to_period('M').value_counts().sort_index()
    for period, count in month_counts.head(10).items():
        print(f"  {period}: {count} records")
        
else:
    print("\nNo EVENT_DATETIME values found. Will use WEEKDAY column for day-of-week matching.")

Finding date range from files with delay data:
Files checked: 504
Files with valid dates: 0
Total date records found: 0

No EVENT_DATETIME values found. Will use WEEKDAY column for day-of-week matching.


In [46]:
# Check the actual format of EVENT_DATETIME values
print("Checking actual EVENT_DATETIME format:")
print("=" * 70)

found_sample = False

for idx, row in SWR_available_stations.iterrows():
    if found_sample:
        break
        
    stanox = str(row['STANOX'])
    tiploc = row['TIPLOC']
    
    station_dir = os.path.join(processed_data_dir, stanox)
    
    if os.path.exists(station_dir):
        parquet_files = [f for f in os.listdir(station_dir) if f.endswith('.parquet')]
        
        for pq_file in parquet_files:
            try:
                preprocessed_data = pd.read_parquet(os.path.join(station_dir, pq_file), engine='fastparquet')
                
                if 'EVENT_DATETIME' in preprocessed_data.columns:
                    # Get non-null EVENT_DATETIME values
                    non_null_dates = preprocessed_data['EVENT_DATETIME'].dropna()
                    
                    if len(non_null_dates) > 0:
                        print(f"Found non-null EVENT_DATETIME in: {tiploc}/{pq_file}")
                        print(f"Number of non-null values: {len(non_null_dates)}")
                        print(f"Data type: {non_null_dates.dtype}")
                        print(f"\nSample values:")
                        for i, val in enumerate(non_null_dates.head(10)):
                            print(f"  [{i}] {val} (type: {type(val)})")
                        
                        # Try to understand the format
                        sample_val = non_null_dates.iloc[0]
                        print(f"\nFirst value details:")
                        print(f"  Value: {sample_val}")
                        print(f"  Type: {type(sample_val)}")
                        print(f"  String representation: '{str(sample_val)}'")
                        
                        found_sample = True
                        break
                        
            except Exception as e:
                print(f"Error reading {pq_file}: {e}")

if not found_sample:
    print("No non-null EVENT_DATETIME values found in any file!")

Checking actual EVENT_DATETIME format:
Found non-null EVENT_DATETIME in: ALDRSHT/FR.parquet
Number of non-null values: 899
Data type: object

Sample values:
  [0] 30-MAR-2024 00:03 (type: <class 'str'>)
  [1] 22-JUN-2024 00:09 (type: <class 'str'>)
  [2] 10-AUG-2024 00:15 (type: <class 'str'>)
  [3] 03-AUG-2024 00:19 (type: <class 'str'>)
  [4] 10-FEB-2024 00:22 (type: <class 'str'>)
  [5] 13-APR-2024 00:22 (type: <class 'str'>)
  [6] 10-AUG-2024 00:22 (type: <class 'str'>)
  [7] 20-APR-2024 00:25 (type: <class 'str'>)
  [8] 10-FEB-2024 00:26 (type: <class 'str'>)
  [9] 21-SEP-2024 00:26 (type: <class 'str'>)

First value details:
  Value: 30-MAR-2024 00:03
  Type: <class 'str'>
  String representation: '30-MAR-2024 00:03'


In [47]:
# Check date range with correct format: DD-MMM-YYYY HH:MM
print("Checking preprocessed data date range (correct format with time):")
print("=" * 70)

all_dates_correct = []
files_with_dates_correct = 0

for idx, row in SWR_available_stations.iterrows():
    stanox = str(row['STANOX'])
    station_dir = os.path.join(processed_data_dir, stanox)
    
    if os.path.exists(station_dir):
        parquet_files = [f for f in os.listdir(station_dir) if f.endswith('.parquet')]
        
        for pq_file in parquet_files:
            try:
                preprocessed_data = pd.read_parquet(os.path.join(station_dir, pq_file), engine='fastparquet')
                
                if 'EVENT_DATETIME' in preprocessed_data.columns:
                    # Parse with correct format: DD-MMM-YYYY HH:MM
                    dates = pd.to_datetime(preprocessed_data['EVENT_DATETIME'], format='%d-%b-%Y %H:%M', errors='coerce')
                    valid_dates = dates.dropna()
                    
                    if len(valid_dates) > 0:
                        all_dates_correct.extend(valid_dates.tolist())
                        files_with_dates_correct += 1
                        
                        if files_with_dates_correct >= 50:  # Sample 50 files
                            break
                            
            except Exception as e:
                pass
    
    if files_with_dates_correct >= 50:
        break

print(f"Files with valid dates: {files_with_dates_correct}")
print(f"Total delay records found: {len(all_dates_correct)}")

if all_dates_correct:
    dates_series = pd.Series(all_dates_correct)
    
    print(f"\nPreprocessed data date range:")
    print(f"  Earliest: {dates_series.min().strftime('%d-%b-%Y %H:%M')} ({dates_series.min().strftime('%A')})")
    print(f"  Latest: {dates_series.max().strftime('%d-%b-%Y %H:%M')} ({dates_series.max().strftime('%A')})")
    print(f"  Total span: {(dates_series.max() - dates_series.min()).days} days")
    
    # Check SWR period
    swr_start = pd.to_datetime('2024-12-08')
    swr_end = pd.to_datetime('2025-01-04')
    
    in_swr = dates_series[(dates_series >= swr_start) & (dates_series <= swr_end)]
    
    print(f"\nSWR period: 08-Dec-2024 to 04-Jan-2025")
    print(f"Delay records in SWR period: {len(in_swr)}")
    
    if len(in_swr) > 0:
        print(f"✓ SWR period IS covered in the data!")
        print(f"  Range: {in_swr.min().strftime('%d-%b-%Y')} to {in_swr.max().strftime('%d-%b-%Y')}")
        print(f"\n  → Can use EXACT DATE matching for trains with delays in SWR period")
    else:
        print(f"✗ NO delay records in SWR period")
        if dates_series.max() < swr_start:
            print(f"  → Data ends BEFORE SWR period starts")
        elif dates_series.min() > swr_end:
            print(f"  → Data starts AFTER SWR period ends")
    
    print(f"\n  → Will use WEEKDAY matching for all trains (including those without delays)")
    
    # Show monthly distribution
    print(f"\nMonthly distribution of delay records:")
    monthly = dates_series.dt.to_period('M').value_counts().sort_index()
    for month, count in monthly.items():
        print(f"  {month}: {count:,} records")
else:
    print("No valid dates found!")

Checking preprocessed data date range (correct format with time):
Files with valid dates: 50
Total delay records found: 32380

Preprocessed data date range:
  Earliest: 04-Feb-2024 01:15 (Sunday)
  Latest: 05-Jan-2025 00:33 (Sunday)
  Total span: 335 days

SWR period: 08-Dec-2024 to 04-Jan-2025
Delay records in SWR period: 2849
✓ SWR period IS covered in the data!
  Range: 08-Dec-2024 to 03-Jan-2025

  → Can use EXACT DATE matching for trains with delays in SWR period

  → Will use WEEKDAY matching for all trains (including those without delays)

Monthly distribution of delay records:
  2024-02: 3,485 records
  2024-03: 3,088 records
  2024-04: 1,606 records
  2024-05: 1,780 records
  2024-06: 2,380 records
  2024-07: 2,572 records
  2024-08: 2,176 records
  2024-09: 2,777 records
  2024-10: 3,387 records
  2024-11: 4,727 records
  2024-12: 3,957 records
  2025-01: 445 records


In [51]:
# Match and merge SWR passenger loading data with preprocessed train data (OPTIMIZED)
print("Matching SWR passenger data with preprocessed train data:")
print("=" * 70)

# Add Location_STANOX to SWR data if not already present
if 'Location_STANOX' not in SWR_data.columns:
    SWR_data['Location_STANOX'] = SWR_data['Location'].map(tiploc_to_stanox)

# Prepare SWR data for merging
# Create a lookup table with only the columns we need
swr_lookup = SWR_data[[
    'Origin_STANOX',
    'Destination_STANOX', 
    'Location_STANOX',
    'Calendar Day',
    'Formation',
    'Car Count',
    'Standard Seated',
    'Standing',
    'Capacity',
    'First Class Seats',
    'Average Train Load on Departure',
    '% of Seated Capacity',
    '% of Total Capacity',
    'Number of Readings'
]].copy()

# FIX: Convert STANOX columns to int to match preprocessed data
swr_lookup['Origin_STANOX'] = swr_lookup['Origin_STANOX'].astype(int)
swr_lookup['Destination_STANOX'] = swr_lookup['Destination_STANOX'].astype(int)
swr_lookup['Location_STANOX'] = swr_lookup['Location_STANOX'].astype(int)

# Map WEEKDAY to day names for merging
weekday_map = {
    0: 'Monday',
    1: 'Tuesday', 
    2: 'Wednesday',
    3: 'Thursday',
    4: 'Friday',
    5: 'Saturday',
    6: 'Sunday'
}

print(f"\nProcessing {len(SWR_available_stations)} stations...")

# Initialize list to store all DataFrames
all_enriched_data = []
files_processed = 0
total_original_records = 0
total_matched_records = 0

# Process each available SWR station
for idx, row in SWR_available_stations.iterrows():
    stanox = str(row['STANOX'])
    tiploc = row['TIPLOC']
    
    station_dir = os.path.join(processed_data_dir, stanox)
    
    if os.path.exists(station_dir):
        parquet_files = [f for f in os.listdir(station_dir) if f.endswith('.parquet')]
        
        for pq_file in parquet_files:
            try:
                # Read preprocessed data
                preprocessed_data = pd.read_parquet(os.path.join(station_dir, pq_file), engine='fastparquet')
                files_processed += 1
                original_count = len(preprocessed_data)
                total_original_records += original_count
                
                # Add STANOX column from folder name (as int)
                preprocessed_data['STANOX'] = int(stanox)
                preprocessed_data['TIPLOC'] = tiploc
                
                # Check required columns
                if all(col in preprocessed_data.columns for col in 
                       ['PLANNED_ORIGIN_LOCATION_CODE', 'PLANNED_DEST_LOCATION_CODE', 'WEEKDAY']):
                    
                    # Map WEEKDAY to day names
                    preprocessed_data['DayName'] = preprocessed_data['WEEKDAY'].map(weekday_map)
                    
                    # OPTIMIZED: Use pandas merge instead of row-by-row matching
                    # Merge on: Origin, Destination, Location, and Day of Week
                    enriched = preprocessed_data.merge(
                        swr_lookup,
                        left_on=['PLANNED_ORIGIN_LOCATION_CODE', 'PLANNED_DEST_LOCATION_CODE', 'STANOX', 'DayName'],
                        right_on=['Origin_STANOX', 'Destination_STANOX', 'Location_STANOX', 'Calendar Day'],
                        how='inner'  # Only keep matches
                    )
                    
                    # Remove duplicate merge columns
                    enriched = enriched.drop(columns=['Origin_STANOX', 'Destination_STANOX', 'Location_STANOX', 'Calendar Day'], errors='ignore')
                    
                    if len(enriched) > 0:
                        # If there are duplicate matches (multiple SWR records for same train),
                        # group by the train's original columns and take mean of SWR columns
                        train_id_cols = [col for col in preprocessed_data.columns if col in enriched.columns]
                        swr_numeric_cols = ['Car Count', 'Standard Seated', 'Standing', 'Capacity', 
                                           'First Class Seats', 'Average Train Load on Departure',
                                           '% of Seated Capacity', '% of Total Capacity', 'Number of Readings']
                        
                        # Aggregate duplicates
                        agg_dict = {}
                        for col in enriched.columns:
                            if col in swr_numeric_cols:
                                agg_dict[col] = 'mean'
                            elif col == 'Formation':
                                agg_dict[col] = 'first'
                            elif col not in train_id_cols:
                                continue
                        
                        if agg_dict:
                            enriched = enriched.groupby(train_id_cols, as_index=False).agg(agg_dict)
                        
                        all_enriched_data.append(enriched)
                        total_matched_records += len(enriched)
                
                # Print progress every 50 files
                if files_processed % 50 == 0:
                    print(f"  Processed {files_processed} files, {total_original_records:,} trains, {total_matched_records:,} matched...")
                
            except Exception as e:
                print(f"  Error processing {pq_file} for {tiploc}: {e}")

# Combine all enriched DataFrames
print(f"\nCombining all enriched data...")
if all_enriched_data:
    enriched_trains_df = pd.concat(all_enriched_data, ignore_index=True)
else:
    enriched_trains_df = pd.DataFrame()

# Summary
print(f"\n{'='*70}")
print(f"Matching Summary:")
print(f"  Files processed: {files_processed}")
print(f"  Total trains in preprocessed data: {total_original_records:,}")
print(f"  Trains matched with SWR data: {total_matched_records:,}")
if total_original_records > 0:
    print(f"  Match rate: {total_matched_records/total_original_records*100:.2f}%")

if len(enriched_trains_df) > 0:
    print(f"\nEnriched Dataset Info:")
    print(f"  Total columns: {len(enriched_trains_df.columns)}")
    print(f"  Total rows: {len(enriched_trains_df):,}")
    print(f"  Memory usage: {enriched_trains_df.memory_usage(deep=True).sum() / 1024**2:.1f} MB")
    
    # Check SWR columns were added
    swr_columns_to_check = ['Formation', 'Car Count', 'Standard Seated', 'Standing', 'Capacity',
                            'First Class Seats', 'Average Train Load on Departure', 
                            '% of Seated Capacity', '% of Total Capacity', 'Number of Readings']
    added_cols = [col for col in swr_columns_to_check if col in enriched_trains_df.columns]
    print(f"\n  SWR columns successfully added: {len(added_cols)}/{len(swr_columns_to_check)}")
    for col in added_cols:
        print(f"    - {col}")
    
    # Show sample statistics
    print(f"\nSample Passenger Loading Statistics:")
    if 'Average Train Load on Departure' in enriched_trains_df.columns:
        print(f"  Average train load: {enriched_trains_df['Average Train Load on Departure'].mean():.1f} passengers")
        print(f"  Min train load: {enriched_trains_df['Average Train Load on Departure'].min():.1f}")
        print(f"  Max train load: {enriched_trains_df['Average Train Load on Departure'].max():.1f}")
    
    if '% of Seated Capacity' in enriched_trains_df.columns:
        print(f"  Average % seated capacity: {enriched_trains_df['% of Seated Capacity'].mean():.2%}")
    
    # Show sample data
    print(f"\nSample of enriched data (first 5 rows):")
    display_cols = ['TIPLOC', 'PLANNED_ORIGIN_LOCATION_CODE', 'PLANNED_DEST_LOCATION_CODE', 
                    'DayName', 'Average Train Load on Departure', '% of Seated Capacity', 'Capacity']
    available_display_cols = [col for col in display_cols if col in enriched_trains_df.columns]
    print(enriched_trains_df[available_display_cols].head().to_string(index=False))
    
    print(f"\n✓ Enriched dataset created successfully as 'enriched_trains_df'")
else:
    print("\n✗ No trains were matched with SWR data")

Matching SWR passenger data with preprocessed train data:


ValueError: cannot convert float NaN to integer